In [152]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from windHelper import WindHelper
from factorModel.factorValidator import *
from factorModel.factorEvent import *
from datetime import *
beginDate = datetime(2007, 1, 1)
endDate = datetime(2017, 8, 22)

# 获取因子的序列

In [172]:
para = "close"
codeList=["CRBSA.RB"]
factor_df = WindHelper.getMultiTimeSeriesDataFrame(codeList=codeList, beginDate=beginDate, endDate=endDate, para=para)
factor_df = factor_df.dropna()
# 日度数据转为月度数据
factor_df = factor_df.resample("M").last()
factor_series = factor_df[(codeList[0]).lower() + "_close"]
factor_series.head()

trade_date
2007-01-31    365.60
2007-02-28    375.86
2007-03-31    389.20
2007-04-30    396.41
2007-05-31    409.82
Freq: M, Name: crbsa.rb_close, dtype: float64

# 获取资产价格序列

In [173]:
para = "close"
codeList=["060E.CS"]
asset_df = WindHelper.getMultiTimeSeriesDataFrame(codeList=codeList, beginDate=beginDate, endDate=endDate, para=para)
asset_df = asset_df.dropna()
asset_series = asset_df[(codeList[0]).lower() + "_" + para]
asset_series.head()

trade_date
2007-01-04    118.1326
2007-01-05    118.3618
2007-01-08    118.3045
2007-01-09    118.3340
2007-01-10    118.2452
Name: 060e.cs_close, dtype: float64

# 观察7-10年国开财富指数和CRB的关系

In [174]:
import pandas_highcharts
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt
df = asset_df.join(factor_df, how="left").fillna(inplace=False, method="ffill")
df.dropna()
display_charts(df, chart_type="stock",secondary_y='crbsa.rb_close', title="CRB测试")

# 夏普比率校验

In [175]:
config = {"relation": NEGATIVE, "type":UP, "his_count": 12, "std_count":1, "period": 1}

In [170]:
report = sharp_ratio_test(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e001", 
                             config=config, lag=1)

In [171]:
import pandas_highcharts
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt
print report['excess_sp']
display_charts(report["ts"][["base","value"]], chart_type="stock", title="CRB 夏普比率测试")

-0.236869416588


# 信号PAE校验

## 事件001

In [117]:
config_e001 = {"relation": NEGATIVE, "type":UP, "his_count": 12, "std_count":2, "period": 1}

In [118]:
report = perform_after_event(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e001", 
                             config=config_e001,
                             asset_price=True,lag=1)

In [119]:
%matplotlib inline
print u"信息比率:"  + str(report["ir"])
print u"收益均值:" + str(report["ts"]["ret"].mean())
print u"准确率:" + str((report["ts"]["ret"]<0).sum()*1.0/report["ts"]["ret"].count())
print u"事件数:" + str(len(report["ts"]))
print u"占样本比例:" + str(len(report["ts"])*1.0/len(report["origin_ts"])*100) +u"%"
print u"样本上涨概率" + str((report["origin_ts"]["ret"]>0).sum()*1.0/report["origin_ts"]["ret"].count())
report["ts"]

信息比率:0.199752558856
收益均值:0.000371813136957
准确率:0.39175257732
事件数:97
占样本比例:11.0730593607%
样本上涨概率0.566210045662


,crbsa.rb_close,ret
2014-02-17,1.091021,0.000202
2014-02-19,1.837086,0.000900
2014-02-20,1.052691,0.001293
2014-02-21,0.532170,0.003731
2014-02-25,0.963799,0.002057
2014-02-27,2.004359,0.001647
2014-03-03,0.861194,-0.004573
2014-03-04,0.479736,-0.003806
2014-03-05,0.309042,-0.001601
2014-03-20,0.003783,0.000424


## 事件003x2

In [57]:
config_e003x2 = {"relation": NEGATIVE, "type":UP, "his_count": 3, "threshold": -0.00, "threshold2": 0.01, "period": 1}

In [58]:
report = perform_after_event(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e003x2", 
                             config=config_e003x2,
                             asset_price=True)

In [59]:
%matplotlib inline
print u"信息比率:"  + str(report["ir"])
print u"收益均值:" + str(report["ts"]["ret"].mean())
print u"准确率:" + str((report["ts"]["ret"]<0).sum()*1.0/report["ts"]["ret"].count())
print u"事件数:" + str(len(report["ts"]))
print u"占样本比例:" + str(len(report["ts"])*1.0/len(report["origin_ts"])*100) +u"%"
print u"样本上涨概率" + str((report["origin_ts"]["ret"]>0).sum()*1.0/report["origin_ts"]["ret"].count())
report["ts"]

信息比率:nan
收益均值:nan


ZeroDivisionError: float division by zero

## 事件006

In [74]:
config_e006 = {"relation": NEGATIVE, "type":UP, "long_count": 180, "short_count":10, "std_count":0, "period": 5}

In [75]:
report = perform_after_event(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e006", 
                             config=config_e006,
                             asset_price=True)

In [76]:
%matplotlib inline
print u"信息比率:"  + str(report["ir"])
print u"收益均值:" + str(report["ts"]["ret"].mean())
print u"准确率:" + str((report["ts"]["ret"]<0).sum()*1.0/report["ts"]["ret"].count())
print u"事件数:" + str(len(report["ts"]))
print u"占样本比例:" + str(len(report["ts"])*1.0/len(report["origin_ts"])*100) +u"%"
print u"样本上涨概率" + str((report["origin_ts"]["ret"]>0).sum()*1.0/report["origin_ts"]["ret"].count())
report["ts"]

信息比率:-0.0579824081546
收益均值:-0.000303214789128
准确率:0.460317460317
事件数:63
占样本比例:44.6808510638%
样本上涨概率0.574468085106


,crbsa.rb_close,ret
2016-03-23,0.767556,0.004522
2016-03-30,6.340667,-0.002226
2016-04-07,8.119056,-0.004264
2016-04-14,9.053111,0.000996
2016-04-21,14.540778,-0.006815
2016-04-28,20.544056,0.001202
2016-05-06,22.075611,0.004002
2016-05-13,20.750389,-0.001096
2016-05-20,20.738722,-0.000249
2016-05-27,19.429278,0.000499


In [ ]:
report